<a href="https://colab.research.google.com/github/SJongH/GroupA/blob/main/Week1_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
def timer(func):
  def wrapper(*args, **kwargs):
    start_time  = time.time()
    result = func (*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Execution time of {func.__name__} {computation_time} seconds")
    return result
  return wrapper

In [ ]:
# Data Generation

import numpy as np

true_b = 1
true_w = 2
N = 100

np.random.seed(42)
x = np.random.rand(N, 1)
epsilon = (0.1 * np.random.randn(N, 1))
y = true_b + true_w * x + epsilon
# * : dot product, @ : matrix multiplication

In [ ]:
# Train-Validation Split

idx = np.arange(N)
split_idx = int(0.8 * N) # 4:1로 나누는게 국룰
train_idx = idx[:split_idx]
val_idx = idx[split_idx:]

x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# scikit-learn 패키지로 쉽게 가능

In [ ]:
@timer
def train_model_numpy(lr = 0.1, epochs = 1000):
  b = np.random.randn(1)
  w = np.random.randn(1)

  for epoch in range(epochs):
    y_hat = b+w*x_train
    error = (y_hat - y_train)
    mse_loss = np.mean(error ** 2)


    b_grad = 2 * np.mean(error)
    w_grad = 2 * np.mean(x_train * error)
    b = b - lr * b_grad
    w = w - lr * w_grad
  return b, w


In [ ]:
b, w = train_model_numpy()

print('b_estimate: {}, w_estimate: {}'.format(b, w))

Execution time of train_model_numpy 0.04592108726501465 seconds
b_estimate: [1.02341381], w_estimate: [1.93680715]


In [ ]:
# Data Generation with Pytorch

import torch
# create tensor in cpu
x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)

# create tensor in gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)

In [ ]:
@timer
def train_model_torch(lr = 0.1, epochs = 1000):
  b = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  w = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)

  for epoch in range(epochs):
    y_hat = b + w * x_train_tensor
    error = (y_hat - y_train_tensor)
    mse_loss = torch.mean(error**2)
    mse_loss.backward()

    # in-place operation 을 이용하므로 no_grad로 사용해야함
    # 일정한 id를 유지하기위해 in-place operation을 사용함

    with torch.no_grad():
      b -= lr * b.grad
      w -= lr * w.grad
    # _: in-place operation 사용하라는 규칙
    b.grad.zero_()
    w.grad.zero_()
  return b, w

In [ ]:
b, w = train_model_torch()
print('b_estimate: {}, w_estimate: {}'.format(b.item(), w.item()))

Execution time of train_model_torch 0.1867084503173828 seconds
b_estimate: 1.0234160423278809, w_estimate: 1.936802864074707


In [ ]:
import torch.optim as optim
import torch.nn as nn

def train_model_torch(lr = 0.1, epochs = 1000):
  b = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  w = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  parameters = [b, w]
  optimizer = optim.SGD(parameters, lr = lr)
  mse_loss = nn.MSELoss()

  for epoch in range(epochs):
    y_hat = b + w*x_train_tensor
    loss = mse_loss(y_hat, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return b, w




In [ ]:
b, w = train_model_torch()
print('b_estimate: {}, w_estimate: {}'.format(b.item(), w.item()))

Execution time of train_model_torch 0.2811307907104492 seconds
b_estimate: 1.0234160423278809, w_estimate: 1.936802864074707


# Quiz 1
- test data 에서 MSE eroor 계산 코드 구현


In [ ]:
x_val_tensor = torch.as_tensor(x_val).to(device)
y_val_tensor = torch.as_tensor(y_val).to(device)

In [ ]:
y_estimate = b+ w*x_val_tensor
mseloss = nn.MSELoss()
error = mseloss(y_val_tensor, y_estimate)
print(error)

tensor(0.0098, dtype=torch.float64, grad_fn=<MseLossBackward0>)


# Quiz 2
- 데이터셋 fitting 하는 함수의 parameter 추정

In [ ]:
import pandas as pd
data = pd.read_pickle('/content/quiz_data.pkl')

In [ ]:
N = int(len(data['x']))
idx = np.arange(N)
split_idx = int(0.8 * N)
train_idx = idx[:split_idx]
test_idx = idx[split_idx:]

In [ ]:
x_train_tensor = torch.as_tensor(data['x'][train_idx]).to(device)
y_train_tensor = torch.as_tensor(data['y'][train_idx]).to(device)
x_test_tensor = torch.as_tensor(data['x'][test_idx]).to(device)
y_test_tensor = torch.as_tensor(data['y'][test_idx]).to(device)

In [ ]:
@timer
def train_model(lr = 0.3, epochs = 1500):
  w2 = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  w = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  b = torch.randn(1, requires_grad = True, dtype = torch.float, device = device)
  parameters = [w, w2, b]
  optimizer = optim.SGD(parameters, lr=lr)
  mseloss = nn.MSELoss()
  for epoch in range(epochs):
    y_hat = w * x_train_tensor+w2*(x_train_tensor**2) + b
    loss = mseloss(y_hat, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return w, w2, b



In [ ]:
def estimate(w, w2, b):
  y_estimate = b+ w*x_val_tensor +w2*(x_val_tensor**2)
  mseloss = nn.MSELoss()
  error = mseloss(y_test_tensor, y_estimate)
  return error

In [ ]:
errors = []
parameters = []
for i in range(10):
  w, w2, b = train_model()
  parameters.append((w,w2, b))
  error = estimate(w,w2, b)
  errors.append(error)

print(errors)

Execution time of train_model 0.4902799129486084 seconds
Execution time of train_model 0.5033102035522461 seconds
Execution time of train_model 0.47930073738098145 seconds
Execution time of train_model 0.4897925853729248 seconds
Execution time of train_model 0.5152542591094971 seconds
Execution time of train_model 0.5087814331054688 seconds
Execution time of train_model 0.48183441162109375 seconds
Execution time of train_model 0.49210405349731445 seconds
Execution time of train_model 0.49263620376586914 seconds
Execution time of train_model 0.49375057220458984 seconds
[tensor(0.0755, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0785, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0771, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0776, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0768, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0764, dtype=torch.float64, grad_fn=<MseLossBackward0>), tensor(0.0768, dtype=torch.float64